In [ ]:
#my streaming try 


In [3]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *




In [4]:
spark = SparkSession.builder \
    .appName("KafkaStreamingExample5") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()



In [5]:

# Kafka connection details
bootstrap_servers = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
kafka_topic = "farid_topic" # add topic name
kafka_username = "JUKQQM4ZM632RECA"
kafka_password = "UUkrPuSttgOC0U9lY3ZansNsKfN9fbxZPFwrGxudDrfv+knTD4rCwK+KdIzVPX0D"

In [7]:
from pyspark.sql.functions import from_json, col, when, to_date, date_format

# Define schema for the incoming JSON data
# Define schema based on the producer's data generating part
schema = StructType([
    StructField("eventType", StringType(), True),
    StructField("customerId", StringType(), True),
    StructField("productId", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("totalAmount", DoubleType(), True),
    StructField("paymentMethod", StringType(), True),
    StructField("recommendedProductId", StringType(), True),
    StructField("algorithm", StringType(), True),
    StructField("metadata", StructType([
        StructField("category", StringType(), True),
        StructField("source", StringType(), True)
    ]), True)
])


# Read data from Kafka topic as a streaming DataFrame
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{kafka_username}" password="{kafka_password}";') \
    .load()

json_df = df.selectExpr("CAST(value AS STRING)").select(from_json("value", schema).alias("data")).select("data.*")

# Handle schema based on event type and derive date and hour columns
good_df = json_df \
    .withColumn("timestamp", col("timestamp").cast("timestamp")) \
    .withColumn("quantity", when(col("eventType") == "purchase", col("quantity"))
                .when(col("eventType") == "addToCart", col("quantity"))
                .otherwise(0)) \
    .withColumn("totalAmount", when(col("eventType") == "purchase", col("totalAmount"))
                .otherwise(0.0)) \
    .withColumn("paymentMethod", when(col("eventType") == "purchase", col("paymentMethod"))
                .otherwise("Unknown")) \
    .withColumn("recommendedProductId", when(col("eventType") == "recommendationclick", col("recommendedProductId"))
                .otherwise("None")) \
    .withColumn("algorithm", when(col("eventType") == "recommendationClick", col("algorithm"))
                .otherwise("None")) \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumn("hour", date_format(col("timestamp"), "HH"))


In [8]:

# Show the resulting DataFrame schema (for debugging purposes)
good_df.printSchema()

root
 |-- eventType: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- productId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- totalAmount: double (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- recommendedProductId: string (nullable = true)
 |-- algorithm: string (nullable = true)
 |-- metadata: struct (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- source: string (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: string (nullable = true)



In [10]:
# Store data on HDFS
output_path = "/data/spark_project/streamingNew1"
query = good_df.writeStream \
    .format("parquet") \
    .option("path", output_path) \
    .option("checkpointLocation", "/data/spark_project/checkpointingNew1") \
    .start()
query.awaitTermination()
spark.stop()

KeyboardInterrupt: 

In [11]:
data = spark.read.parquet("/data/spark_project/streamingNew1/*")


In [14]:
from pyspark.sql.functions import avg

average_total_amount = data.groupBy("paymentMethod").agg(avg("totalAmount"))
average_total_amount.show()

+-------------+----------------+
|paymentMethod|avg(totalAmount)|
+-------------+----------------+
|      Unknown|             0.0|
|       PayPal|          436.84|
|   Debit Card|          303.46|
+-------------+----------------+



In [16]:
total_events_by_type = data.groupBy("eventType").count()
total_events_by_type.show()



+-------------------+-----+
|          eventType|count|
+-------------------+-----+
|           purchase|    2|
|          addToCart|    9|
|        productView|    7|
|recommendationClick|   11|
+-------------------+-----+

